In [49]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
import scipy as sp
from bayes_opt import BayesianOptimization
from sklearn.feature_selection import SelectFromModel

In [3]:
train_test_all=pd.read_csv('train_test_train_all.csv')#样本滑窗特征+乘除特征

In [60]:
train_test_all['power_generation'].fillna(0,inplace=True)

In [4]:
train_x=train_test_all[:8500]
train_y=train_test_all['power_generation'][:8500]
val_x=train_test_all[8500:9000]
val_y=train_test_all['power_generation'][8500:9000]

# PolynomialFeatures

In [31]:
#初步测试
train_data=pd.read_csv('train_data_li.csv')#沥川清洗数据
test_data=pd.read_csv('test_data_li.csv')

In [72]:
def generate_train_data(train_data, test_data, poly=False, select=False):

    y = train_data['power_generation']
    X = train_data.drop(['power_generation','ID'], axis=1)
    sub_data = test_data.drop(['power_generation','ID'], axis=1)

    if poly:
        from sklearn.preprocessing import PolynomialFeatures
        poly = PolynomialFeatures(degree=3, interaction_only=True)
        X = poly.fit_transform(X)
        sub_data = poly.transform(sub_data)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    if select:
        from sklearn.feature_selection import SelectFromModel
        sm = SelectFromModel(GradientBoostingRegressor(random_state=2))
        X_train = sm.fit_transform(X_train, y_train)
        X_test = sm.transform(X_test)
        #sub_data = sm.transform(sub_data)
        sub_data = sub_data
    return X_train, X_test, y_train, y_test, sub_data

In [33]:
X_train, X_test, y_train, y_test, sub_data = generate_train_data(train_data, test_data, poly=True, select=False)
print(X_train.shape, sub_data.shape)

(7200, 1160) (8409, 1160)


In [34]:
import lightgbm as lgb

In [35]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_test = lgb.Dataset(X_test, y_test)

In [77]:
def lgb_eval(num_leaves,
             min_data,
             sub_feature
            ):

    params['num_leaves'] = int(round(num_leaves))
    params['min_data'] = int(min_data)
    params['sub_feature'] = max(min(sub_feature, 1), 0)
    
    gbm = lgb.train(params, lgb_train, 4000)
    
    pred = gbm.predict(X_test)
    rmsetmp = sp.sqrt(sp.mean((y_test - pred) ** 2))
    score = 1 / (1 + rmsetmp)
    return score

In [78]:
lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (20, 200),
                                        'min_data': (10, 80),
                                        'sub_feature': (0.3, 1)
                                       })

In [39]:
if __name__ == '__main__':
    num_rounds = 4000
    random_state = 42
    num_iter = 25
    init_points = 5
    params = {
        'eta': 0.1,
        'learning_rate': 0.002,
        'objective': 'regression',
        'boosting_type': 'gbdt',
        'metric': 'rmse',
        'min_hessian': 1,
        'verbose': -1
    }

In [40]:
lgbBO.maximize(init_points=init_points, n_iter=num_iter)

Initialization
-------------------------------------------------------------------------
 Step |   Time |      Value |   min_data |   num_leaves |   sub_feature | 
    1 | 07m51s |    0.88802 |    66.8344 |      91.0217 |        0.3257 | 
    2 | 17m02s |    0.89072 |    40.8215 |     190.6857 |        0.9329 | 
    3 | 05m23s |    0.89163 |    50.8637 |      37.4146 |        0.7165 | 
    4 | 08m05s |    0.89189 |    79.8432 |     156.4945 |        0.7154 | 
    5 | 05m15s |    0.89068 |    78.8641 |      32.6934 |        0.9271 | 


/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


Bayesian Optimization
-------------------------------------------------------------------------
 Step |   Time |      Value |   min_data |   num_leaves |   sub_feature | 
    6 | 21m57s |    0.89025 |    25.4883 |     197.2417 |        0.7867 | 
    7 | 04m41s |    0.89033 |    54.6625 |      33.2827 |        0.9193 | 


/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([  5.10649363e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 58, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)


    8 | 16m34s |    0.88827 |    13.8968 |     139.0020 |        0.7967 | 
    9 | 08m21s |    0.88972 |    58.6207 |     177.3780 |        0.4475 | 
   10 | 07m35s |    0.89191 |    73.2151 |     153.6133 |        0.6729 | 
   11 | 04m42s |    0.89203 |    31.8450 |      35.0787 |        0.8046 | 


...

/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ -3.83719162e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 52, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)


   27 | 06m08s |    0.89084 |    61.7778 |      57.8931 |        0.5304 | 
   28 | 08m56s |    0.89141 |    68.7409 |     184.4357 |        0.9509 | 


/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00251131]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 54, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)


   29 | 07m33s |    0.89073 |    45.5450 |      72.2492 |        0.5377 | 
   30 | 10m08s |    0.89082 |    52.6778 |     199.7284 |        0.6763 | 


/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00018072]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 59, 'nit': 7, 'warnflag': 2}
  " state: %s" % convergence_dict)
/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00047465]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 57, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)


In [42]:
params = {
        'eta': 0.1,
        'learning_rate': 0.002,
        'objective': 'regression',
        'boosting_type': 'gbdt',
        'metric': 'rmse',
        'min_hessian': 1,
        'verbose': -1,
        'num_leaves':146,
        'min_data':78,
        'sub_feature': 0.8468 
    
    }
gbm = lgb.train(params, lgb_train, 4000)

In [43]:
pred = gbm.predict(sub_data)

In [44]:
submit_data=pd.read_csv('submit_example.csv',header=None)
submit_data.columns=['ID','power_generation']

In [45]:
submit_data['power_generation']=pred
submit_data.to_csv('submit/result_0816_lgbm_1.csv',index=False, header=None) 

# 全features

In [61]:
train_data=train_test_all[:9000]
test_data=train_test_all[9000:]

In [73]:
X_train, X_test, y_train, y_test, sub_data = generate_train_data(train_data, test_data, poly=False, select=True)
print(X_train.shape, sub_data.shape)

(7200, 163) (8409, 958)


In [74]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_test = lgb.Dataset(X_test, y_test)

In [76]:
if __name__ == '__main__':
    num_rounds = 4000
    random_state = 42
    num_iter = 25
    init_points = 5
    params = {
        'eta': 0.1,
        'learning_rate': 0.002,
        'objective': 'regression',
        'boosting_type': 'gbdt',
        'metric': 'rmse',
        'min_hessian': 1,
        'verbose': -1
    }

In [79]:
lgbBO.maximize(init_points=init_points, n_iter=num_iter)

Initialization
-------------------------------------------------------------------------
 Step |   Time |      Value |   min_data |   num_leaves |   sub_feature | 
    1 | 01m04s |    0.88575 |    67.0275 |      34.7060 |        0.6918 | 
    2 | 02m03s |    0.88646 |    37.2802 |      76.4406 |        0.5337 | 
    3 | 01m25s |    0.89019 |    30.5506 |      46.1509 |        0.7650 | 
    4 | 01m16s |    0.88743 |    61.0721 |      41.4736 |        0.8209 | 
    5 | 01m50s |    0.88372 |    70.9355 |     176.9829 |        0.5177 | 


/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ -1.18737357e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 49, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)
/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


Bayesian Optimization
-------------------------------------------------------------------------
 Step |   Time |      Value |   min_data |   num_leaves |   sub_feature | 
    6 | 01m22s |    0.88729 |    61.2947 |      41.5753 |        0.7560 | 
    7 | 05m06s |    0.88742 |    16.9204 |     187.6023 |        0.6800 | 
    8 | 04m16s |    0.88447 |    11.1535 |     165.7029 |        0.3888 | 


/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.0002004]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 53, 'nit': 6, 'warnflag': 2}
  " state: %s" % convergence_dict)


    9 | 01m46s |    0.87918 |    72.8182 |     199.9030 |        0.3494 | 
   10 | 04m38s |    0.88512 |    10.3700 |     176.0625 |        0.5982 | 
   11 | 01m42s |    0.88746 |    79.9365 |     144.8422 |        0.8161 | 
   12 | 03m08s |    0.88798 |    22.4580 |     130.6456 |        0.6227 | 
   13 | 03m07s |    0.88755 |    22.7605 |     130.8355 |        0.5838 | 


...

   23 | 03m11s |    0.88403 |    26.3555 |     156.2195 |        0.3536 | 
   24 | 02m57s |    0.88783 |    11.2328 |     115.7406 |        0.6660 | 


/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ -4.30698501e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 55, 'nit': 6, 'warnflag': 2}
  " state: %s" % convergence_dict)
/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00042569]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 48, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)


   25 | 01m44s |    0.88671 |    72.0823 |     183.0759 |        0.8039 | 


/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00508633]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 55, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)


   26 | 01m42s |    0.88627 |    70.8297 |     177.1740 |        0.6199 | 
   27 | 01m09s |    0.88779 |    61.4613 |      39.0917 |        0.7779 | 
   28 | 01m30s |    0.88536 |    78.9464 |      67.2129 |        0.6133 | 
   29 | 02m02s |    0.88424 |    19.1250 |      87.9674 |        0.3782 | 
   30 | 02m09s |    0.88658 |    54.3015 |     135.4356 |        0.5780 | 


In [ ]:
3 | 01m25s |    0.89019 |    30.5506 |      46.1509 |        0.7650 | 
结论：暴力全特征没有ployFeaturexiao